In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [5]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [6]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 100 
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 100
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

In [10]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [11]:
# Check to make sure replacement was successful
print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64


In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [13]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

In [14]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

In [16]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [17]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [19]:
#define the model 
number_input_features = X_train_scaled.shape[1]

In [20]:
# Try different configurations
attempts = [
    {
        'hidden_layers': [128, 64, 32],
        'activation': 'relu',
        'dropout': 0.2,
        'epochs': 100
    },
    {
        'hidden_layers': [256, 128, 64, 32],
        'activation': 'relu',
        'dropout': 0.3,
        'epochs': 150
    },
    {
        'hidden_layers': [128, 128, 64, 64, 32],
        'activation': 'relu',
        'dropout': 0.2,
        'epochs': 200
    }
]

best_model = None
best_accuracy = 0

In [21]:
for attempt in attempts:
    print(f"Attempting model with layers: {attempt['hidden_layers']}, dropout: {attempt['dropout']}, epochs: {attempt['epochs']}")
    
    nn = tf.keras.models.Sequential()
    
    # Input layer and first hidden layer
    nn.add(tf.keras.layers.Dense(units=attempt['hidden_layers'][0], activation=attempt['activation'], input_dim=number_input_features))
    nn.add(tf.keras.layers.Dropout(attempt['dropout']))
    
    # Additional hidden layers
    for units in attempt['hidden_layers'][1:]:
        nn.add(tf.keras.layers.Dense(units=units, activation=attempt['activation']))
        nn.add(tf.keras.layers.Dropout(attempt['dropout']))
    
    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Attempting model with layers: [128, 64, 32], dropout: 0.2, epochs: 100
Attempting model with layers: [256, 128, 64, 32], dropout: 0.3, epochs: 150
Attempting model with layers: [128, 128, 64, 64, 32], dropout: 0.2, epochs: 200


C:\Users\Bootcamp\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
 # Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
 # Train the model
history = nn.fit(X_train_scaled, y_train, epochs=attempt['epochs'], batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

Epoch 1/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6828 - loss: 0.6133 - val_accuracy: 0.7229 - val_loss: 0.5606
Epoch 2/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7246 - loss: 0.5696 - val_accuracy: 0.7227 - val_loss: 0.5582
Epoch 3/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7263 - loss: 0.5638 - val_accuracy: 0.7268 - val_loss: 0.5561
Epoch 4/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7300 - loss: 0.5577 - val_accuracy: 0.7292 - val_loss: 0.5587
Epoch 5/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7291 - loss: 0.5593 - val_accuracy: 0.7283 - val_loss: 0.5567
Epoch 6/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7326 - loss: 0.5533 - val_accuracy: 0.7238 - val_loss: 0.5563
Epoch 7/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7313 - loss: 0.5539 - val_accuracy: 0.7216 - val_loss: 0.5575
Epoch 8/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7245 - loss: 0.5597 - val_accu

In [27]:
  # Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    
if model_accuracy > best_accuracy:
        best_accuracy = model_accuracy
        best_model = nn

215/215 - 0s - 582us/step - accuracy: 0.7296 - loss: 0.5510
Loss: 0.5510192513465881, Accuracy: 0.7295918464660645


In [28]:
# Export the best model to HDF5 file
best_model.save('AlphabetSoupCharity_Optimization.h5')

# Print the best accuracy
print(f"Best Model Accuracy: {best_accuracy}")

Best Model Accuracy: 0.7295918464660645
